# Assignment #1: Building an inverted index
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program that collects all the words from a set of documents
* Build an index from the words
* Represent a document using the Tf.Idf values
* Write a short report of 1 to 2 pages on the assignment
* Read a description of an industrial system and answer a question on it
* Discuss the potential of corpora in applications

## Submission

When you have written all the missing code and run all the cells, you will submit your notebook to an automatic marking system. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.

## Description of the assignment

### Outline

In this lab, you will build an indexer to index all the words in a corpus. Conceptually, an index consists of rows with one word per row and the list of files and positions, where this word occurs. Such a row is called a _posting list_. You will encode the position of a word by the number of characters from the start of the file.
<pre>
word1: file_name pos1 pos2 pos3... file_name pos1 pos2 ...
word2: file_name pos1 pos2 pos3... file_name pos1 pos2 ...
...
</pre>

#### Imports

Some imports. Add others as needed

In [155]:
import bz2
import math
import os
import pickle
from collections import Counter
import regex as re
import requests
import sys
import time
from zipfile import ZipFile

## Corpus

You will create an index for a corpus of Selma Lagerlöf's works: To gather the corpus, you can alternatively:
1. Download the <a href="https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip">Selma folder</a> and uncompress it. It contains novels by <a href="https://sv.wikipedia.org/wiki/Selma_Lagerl%C3%B6f">Selma Lagerlöf</a>. The text of these novels was extracted from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at <a href="https://litteraturbanken.se/">Litteraturbanken</a>.
2. Or run this cell that will download the corpus and place it in your folder.

In [19]:
# Parameters for Selma dataset
SELMA_URL = "https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip"

SELMA_FILES = [
    os.path.join("Selma", fname) 
    for fname in 
    [
        "bannlyst.txt", 
        "gosta.txt", 
        "herrgard.txt", 
        "jerusalem.txt", 
        "kejsaren.txt", 
        "marbacka.txt", 
        "nils.txt", 
        "osynliga.txt", 
        "troll.txt"
    ]
]

def download_and_extract_selma():
    """Downloads and unpacks Selma.zip"""
    
    # Download if not all files exist
    req = requests.get(SELMA_URL, stream=True)
    if req.status_code != 200:
        print("Failed to download file, got status: " + req.status_code)
        req.close()
    else:
        with open("Selma.zip", "wb") as fd:
            written = 0
            for chunk in req.iter_content(chunk_size=65536):
                fd.write(chunk)
                written += len(chunk)
                print("Downloading: %d bytes written to Selma.zip" % written)

        print("Selma.zip donwnloaded.")
        req.close()
        
        selma_zipfile = ZipFile("Selma.zip")
        selma_files_to_extract = [zi for zi in selma_zipfile.filelist if not zi.filename.startswith("__") and zi.filename.endswith(".txt")]
        for zi in selma_files_to_extract:
            selma_zipfile.extract(zi)
            print("Extracted: " + zi.filename)
            
        print("Done!")
        
# If not all path exists (all are true), then download
if not all([os.path.exists(fname) for fname in SELMA_FILES]):
    download_and_extract_selma()
else:
    print("Selma has been downloaded.")
    
SELMA_FILES

Downloading: 65536 bytes written to Selma.zip
Downloading: 131072 bytes written to Selma.zip
Downloading: 196608 bytes written to Selma.zip
Downloading: 262144 bytes written to Selma.zip
Downloading: 327680 bytes written to Selma.zip
Downloading: 393216 bytes written to Selma.zip
Downloading: 458752 bytes written to Selma.zip
Downloading: 524288 bytes written to Selma.zip
Downloading: 589824 bytes written to Selma.zip
Downloading: 655360 bytes written to Selma.zip
Downloading: 720896 bytes written to Selma.zip
Downloading: 786432 bytes written to Selma.zip
Downloading: 851968 bytes written to Selma.zip
Downloading: 917504 bytes written to Selma.zip
Downloading: 983040 bytes written to Selma.zip
Downloading: 1048576 bytes written to Selma.zip
Downloading: 1114112 bytes written to Selma.zip
Downloading: 1179648 bytes written to Selma.zip
Downloading: 1245184 bytes written to Selma.zip
Downloading: 1310720 bytes written to Selma.zip
Downloading: 1376256 bytes written to Selma.zip
Download

['Selma/bannlyst.txt',
 'Selma/gosta.txt',
 'Selma/herrgard.txt',
 'Selma/jerusalem.txt',
 'Selma/kejsaren.txt',
 'Selma/marbacka.txt',
 'Selma/nils.txt',
 'Selma/osynliga.txt',
 'Selma/troll.txt']

### Running the indexer (optional)

In a production context, your final program would take a corpus as input (here the Selma Lagerlöf's novels) and create an index of all the words with their positions. You should be able to run it this way:
<pre>$ python indexer.py folder_name</pre>
In this lab, you will write the index in a Jupyter Notebook. The conversion into a Python program is left as an optional exercise.

## Programming the Indexer

To make programming easier, you will split this exercise into five steps:
1. Index one file;
2. Read the content of a folder
3. Create a master index for all the files
4. Use tfidf to represent the documents (novels)
5. Compare the documents of a collection

You will use dictionaries to represent the postings.

### Indexing one file

#### Description

<p>Write a program that reads one document <tt>file_name.txt</tt> and outputs an index file:
            <tt>file_name.idx</tt>:
        </p>
        <ol>
            <li>The index file will contain all the unique words in the document,
                where each word is associated with the list of its positions in the document.
            </li>
            <li>You will represent this index as a dictionary, where the keys will be the words, and
                the values, the lists of positions
            </li>
            <li>As words, you will consider all the strings of letters that you will set in lower case.
                You will not index the rest (i.e. numbers, punctuations, or symbols).
            </li>
            <li>To extract the words, use Unicode regular expressions. Do not use <tt>\w+</tt>,
                for instance, but the Unicode equivalent.
            </li>
            <li>The word positions will correspond to the number of characters from the beginning of the file.
                (The word offset from the beginning)
            </li>
            <li>You will use the <tt>finditer()</tt> method to find the positions of the words.
                This will return you match objects,
                where you will get the matches and the positions with
                the <tt>group()</tt> and <tt>start()</tt> methods.
            </li>
            <li>You will use the pickle package to write your dictionary in an file,
                see <a href="https://wiki.python.org/moin/UsingPickle">https://wiki.python.org/moin/UsingPickle</a>.
            </li>
        </ol>

Below is an excerpt of the index of the `bannlyst.txt` text for the words <i>gjord</i>, <i>uppklarnande</i>, and <i>stjärnor</i>. The data is stored in a dictionary:

<pre>
{...
'gjord': [8600, 183039, 220445],
'uppklarnande': [8617],
'stjärnor': [8641], ...
}
</pre>
where the word <i>gjord</i> occurs three times in the text at positions 8600, 183039, and 220445, <i>uppklarnande</i>, once at position 8617, and <i>stjärnor</i>, once at position 8641.

#### Writing a tokenizer 

Write a **Unicode** regular expression to find words defined as sequences of letters. Using the Unicode notation is compulsory. If you are not sure, please refer to the lecture.

In [31]:
# Write your regex here
regex = "\\p{L}+"

In [32]:
re.findall(regex, 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa')

['En',
 'gång',
 'hade',
 'de',
 'på',
 'Mårbacka',
 'en',
 'barnpiga',
 'som',
 'hette',
 'Back',
 'Kajsa']

Using `regex`, write `tokenize(text)` function to tokenize a text. Return their positions.

In [82]:
# Write your code here
def tokenize(text):
    return [(m.group(), m.span()) for m in re.finditer(regex, text)]


In [83]:
tokens = tokenize('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.')
list(tokens)

[('En', (0, 2)),
 ('gång', (3, 7)),
 ('hade', (8, 12)),
 ('de', (13, 15)),
 ('på', (16, 18)),
 ('Mårbacka', (19, 27)),
 ('en', (28, 30)),
 ('barnpiga', (31, 39)),
 ('som', (41, 44)),
 ('hette', (45, 50)),
 ('Back', (51, 55)),
 ('Kajsa', (56, 61))]

#### Extracting indices

Write a `text_to_idx(words)` function to extract the indices from the list of tokens (words). Return a dictionary, where the keys will be the tokens (words), and the values a list of positions.

In [88]:
# Write your code here
def text_to_idx(words):
    index = {}
    for i, (word, span) in enumerate(words):
        if word not in index:
            index[word] = []
        index[word].append(i)
    return index

In [89]:
tokens = tokenize('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.'.lower().strip())
text_to_idx(tokens)

{'en': [0, 6],
 'gång': [1],
 'hade': [2],
 'de': [3],
 'på': [4],
 'mårbacka': [5],
 'barnpiga': [7],
 'som': [8],
 'hette': [9],
 'back': [10],
 'kajsa': [11]}

#### Reading one file

Read one file, _Mårbacka_, `marbacka.txt`, set it in lowercase, tokenize it, and index it. Call this index `idx`

In [96]:
# Write your code here
with open("marbacka.txt", "r", encoding="utf-8") as f:
    marback = f.read()
tokens = tokenize(marback.lower().strip())
idx = text_to_idx(tokens)

In [97]:
print(idx['mårbacka'])

[2, 15, 124, 304, 461, 603, 2777, 2828, 5047, 7611, 8840, 9053, 9347, 10806, 11356, 11638, 12059, 12086, 12167, 12170, 12421, 12811, 12950, 13312, 13331, 13391, 13766, 13817, 13861, 13926, 13973, 13997, 14106, 14452, 14659, 14696, 15096, 15174, 15256, 16090, 17433, 17723, 18068, 18896, 18944, 19019, 19546, 19622, 20298, 20469, 20703, 20817, 20864, 20962, 20998, 22007, 22541, 22830, 22857, 23258, 23663, 23834, 23981, 24695, 24743, 24858, 25095, 25379, 25654, 26462, 27309, 27791, 27930, 27983, 28064, 28166, 28483, 28793, 28847, 29040, 29194, 29836, 30298, 30341, 30450, 30675, 31086, 32821, 32884, 33545, 34119, 34377, 34395, 34764, 35392, 36379, 37042, 37297, 37402, 37668, 37791, 39003, 39418, 39666, 39690, 39743, 39806, 40146, 41078, 41091, 41317, 41441, 41565, 41983, 42310, 42451, 43013, 43323, 43407, 43501, 43520, 43832, 44713, 44970, 45077, 45874, 46569, 46801, 46919, 47170, 47611, 48148, 48202, 48292, 48379, 48508, 48629, 48783, 49056, 49158, 49257, 49466, 52035, 53335, 54459, 54668,

#### Saving the index

Save your index in a file so that you can reuse it. Use the pickle module.

In [ ]:
# Write your code here
index_file = 'marbacka.idx'
with open(index_file, 'wb') as f:
    pickle.dump(idx, f)

Read back your file and store the content in `idx`

In [101]:
# Write your code here
with open(index_file, 'rb') as f:
    idx_loaded = pickle.load(f)

In [13]:
idx['mårbacka']

[16,
 139,
 752,
 1700,
 2582,
 3324,
 15117,
 15404,
 27794,
 42175,
 49126,
 50407,
 52053,
 60144,
 63374,
 64910,
 67182,
 67330,
 67799,
 67824,
 69232,
 71328,
 72099,
 74147,
 74255,
 74614,
 76610,
 76884,
 77138,
 77509,
 77787,
 77936,
 78574,
 80597,
 81782,
 82003,
 84363,
 84786,
 85251,
 89837,
 97093,
 98642,
 100474,
 105063,
 105298,
 105721,
 108710,
 109133,
 112844,
 113725,
 114997,
 115583,
 115833,
 116368,
 116557,
 121896,
 124823,
 126409,
 126542,
 128758,
 130976,
 131939,
 132826,
 136914,
 137187,
 137872,
 139196,
 140721,
 142324,
 146781,
 151497,
 154335,
 155139,
 155438,
 155886,
 156405,
 158108,
 159817,
 160107,
 161158,
 162085,
 165847,
 168316,
 168528,
 169111,
 170333,
 172684,
 182047,
 182427,
 186362,
 189535,
 190999,
 191110,
 193177,
 196686,
 202552,
 206340,
 207789,
 208382,
 209874,
 210525,
 217464,
 219933,
 221393,
 221533,
 221880,
 222213,
 224190,
 229501,
 229598,
 230783,
 231453,
 232140,
 234427,
 236193,
 236950,
 240168,

### Reading the content of a folder

Write a `get_files(dir, suffix)` function that reads all the files in a folder with a specific `suffix` (txt). You will need the Python `os` package, see <a href="https://docs.python.org/3/library/os.html">https://docs.python.org/3/library/os.html</a>. You will return the file names in a list.

You can reuse this function:

In [102]:
def get_files(dir, suffix):
    """
    Returns all the files in a folder ending with suffix
    :param dir:
    :param suffix:
    :return: the list of file names
    """
    files = []
    for file in os.listdir(dir):
        if file.endswith(suffix):
            files.append(file)
    return files

In [110]:
# Write your code here
print(get_files(os.getcwd()+"/corpus", "")) 

['marbacka.txt', 'bannlyst.txt', 'jerusalem.txt', 'gosta.txt', 'nils.txt', 'herrgard.txt', 'osynliga.txt', 'troll.txt', 'kejsaren.txt']


### Creating a master index

Complete your program with the creation of master index, where you will associate each word of the corpus with the files, where it occur and its positions: a posting list
Below is an except of the master index with the words <i>samlar</i> and <i>ände</i>:

In [16]:
{'samlar':
            {'troll.txt': [641880, 654233],
            'nils.txt': [51805, 118943],
            'osynliga.txt': [399121],
            'gosta.txt': [313784, 409998, 538165]},
 'ände':
            {'troll.txt': [39562, 650112],
            'kejsaren.txt': [50171],
            'marbacka.txt': [370324],
            'nils.txt': [1794],
            'osynliga.txt': [272144]}
}

{'samlar': {'troll.txt': [641880, 654233],
  'nils.txt': [51805, 118943],
  'osynliga.txt': [399121],
  'gosta.txt': [313784, 409998, 538165]},
 'ände': {'troll.txt': [39562, 650112],
  'kejsaren.txt': [50171],
  'marbacka.txt': [370324],
  'nils.txt': [1794],
  'osynliga.txt': [272144]}}

The word <i>samlar</i>, for instance, occurs three times in the gosta text at positions
            313784, 409998, and 538165.

In [139]:
# write your code here
corpus_folder = os.path.join(os.getcwd(), "corpus")
corpus_files = get_files(corpus_folder, "")  
master_index = {}

for file_name in corpus_files:
    file_path = os.path.join(corpus_folder, file_name)  # full path to the file
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read().lower()
    
    tokens = tokenize(text)
    
    for word, start_pos in tokens:
        if word not in master_index:
            master_index[word] = {}
        if file_name not in master_index[word]:  # store using just the filename
            master_index[word][file_name] = []
        master_index[word][file_name].append(start_pos)
    


In [18]:
master_index['samlar']

{'troll.txt': [641880, 654233],
 'nils.txt': [51805, 118943],
 'osynliga.txt': [399121],
 'gosta.txt': [313784, 409998, 538165]}

In [19]:
master_index['mårbacka']

{'troll.txt': [226291, 387634, 392959],
 'marbacka.txt': [16,
  139,
  752,
  1700,
  2582,
  3324,
  15117,
  15404,
  27794,
  42175,
  49126,
  50407,
  52053,
  60144,
  63374,
  64910,
  67182,
  67330,
  67799,
  67824,
  69232,
  71328,
  72099,
  74147,
  74255,
  74614,
  76610,
  76884,
  77138,
  77509,
  77787,
  77936,
  78574,
  80597,
  81782,
  82003,
  84363,
  84786,
  85251,
  89837,
  97093,
  98642,
  100474,
  105063,
  105298,
  105721,
  108710,
  109133,
  112844,
  113725,
  114997,
  115583,
  115833,
  116368,
  116557,
  121896,
  124823,
  126409,
  126542,
  128758,
  130976,
  131939,
  132826,
  136914,
  137187,
  137872,
  139196,
  140721,
  142324,
  146781,
  151497,
  154335,
  155139,
  155438,
  155886,
  156405,
  158108,
  159817,
  160107,
  161158,
  162085,
  165847,
  168316,
  168528,
  169111,
  170333,
  172684,
  182047,
  182427,
  186362,
  189535,
  190999,
  191110,
  193177,
  196686,
  202552,
  206340,
  207789,
  208382,
  2098

Save your master index in a file and read it again

In [140]:
# Write your code here
with open('master.idx', 'wb') as f:
    pickle.dump(master_index, f)

In [142]:
with open('master.idx', 'rb') as f:
    master_loaded = pickle.load(f)
master_loaded['samlar']

{'gosta.txt': [(313784, 313790), (409998, 410004), (538165, 538171)],
 'nils.txt': [(51805, 51811), (118943, 118949)],
 'osynliga.txt': [(399121, 399127)],
 'troll.txt': [(641880, 641886), (654233, 654239)]}

#### Concordances

Write a `concordance(word, master_index, window)` function to extract the concordances of a `word` within a window of `window` characters

In [148]:
# Write your code here
def concordance(word, master_index, window):
    word = word.lower()
    results = []
    if word not in master_index:
        return results
    for file_name, positions in master_index[word].items():
        file_path = os.path.join(corpus_folder, file_name)  # full path
        with open(file_path, 'r', encoding='utf=8') as f:
            
            text = f.read()
            for span in positions:
                start_pos = span[0]
                start = max(start_pos - window, 0)
                end = min(start_pos + len(word)+ window, len(text))
                context = text[start:end]
                results.append((file_name, context))
    return results

In [149]:
concordance('samlar', master_index, 25)


[('gosta.txt', 'om ligger nära Borg, och samlar ihop ett litet middagssä'),
 ('gosta.txt', 'lika förstämda.\n\nMen hon samlar upp allt detta som glöda'),
 ('gosta.txt', 'n ensam i livet.\n\nDärmed samlar han korten tillhopa, res'),
 ('nils.txt', ' bara, att du i all hast samlar ihop så mycket boss och '),
 ('nils.txt', 'ar stannat hemma, och nu samlar de sig för att intränga '),
 ('osynliga.txt', ' till höger i kärran och samlar just ihop tömmarna, och '),
 ('troll.txt', 'en örtkunnig läkare, som samlar in markens växter för at'),
 ('troll.txt', 'älper dem, och medan hon samlar och handlar för deras rä')]

### Representing Documents with tf-idf

Once you have created the index, you will represent each document in your corpus as a dictionary. The keys of these dictionaries will be the words and you will define the value of a word with the tf-idf metric: 
1. Read the description of the tf-idf measure on Wikipedia (<a href="https://en.wikipedia.org/wiki/Tf%E2%80%93idf">https://en.wikipedia.org/wiki/Tf-idf</a>)
2. After reading the description, you probably realized that there are multiple definitions of tf-idf. In this assignment, 
 * Tf will be the relative frequency of the term in the document and 
 * idf, the logarithm base 10 of the inverse document frequency.
        
You have below the tf-idf values for a few words. In our example, the word <i>gås</i> has the value 0 in bannlyst.txt and the value 0.000101001964 in nils.txt

<pre>
troll.txt
	känna	 0.0
	gås	 0.0
	nils	 2.148161748868631e-06
	et	 0.0
kejsaren.txt
	känna	 0.0
	gås	 0.0
	nils	 8.08284798629935e-06
	et	 8.273225429362848e-05
marbacka.txt
	känna	 0.0
	gås	 0.0
	nils	 7.582276564686669e-06
	et	 9.70107989686256e-06
herrgard.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
nils.txt
	känna	 0.0
	gås	 0.00010100196417506702
	nils	 0.00010164426900380124
	et	 0.0
osynliga.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
jerusalem.txt
	känna	 0.0
	gås	 0.0
	nils	 4.968292117670952e-06
	et	 0.0
bannlyst.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
gosta.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
</pre>

Conceptually, the tf-idf representation is a vector. In your program, you will keep this idea and use all the words in the corpus as keys: Each dictionary will include all the words of the corpus as keys. The value of the key is then possibly 0, meaning that the word in not in the document or is in all the documents as for the word `nils` in `gosta.tx`. 

As further work, you may think of optimizing this part.

In [ ]:
import os
import math
import regex
from collections import Counter

vocab = set(master_index.keys())
N = len(corpus_files)

# Document frequency
df = {word: len(posting) for word, posting in master_index.items()}

tfidf = {}
for file_name in corpus_files:
    tfidf[file_name] = {}
    
    file_path = os.path.join(corpus_folder, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read().lower()
    
    # Tokenize with Unicode regex
    tokens = [m.group() for m in regex.finditer(r'\p{L}+', text)]
    
    # Count term frequencies
    counts = Counter(tokens)
    
    # Compute TF-IDF
    for token in vocab:
        tf = counts.get(token, 0)       # frequency of token in this doc
        idf = math.log(N / df[token]) if df[token] > 0 else 0
        tfidf[file_name][token] = tf * idf


In [159]:
tfidf['troll.txt']['känna']

0.0

In [160]:
tfidf['troll.txt']['nils']

0.5877866649021191

### Comparing Documents

Using the cosine similarity, compare all the pairs of documents with their tf-idf representation and present your results in a table. You will include this table in your report.

#### Cosine similarity

Write a function computing the cosine similarity between two documents: `cosine_similarity(document1, document2)`

In [161]:
# Write your code here
def cosine_similarity(document1, document2):
    common_words = set(document1.keys()) & set(document2.keys())
    dot_product = sum(document1[word] * document2[word] for word in common_words)
    
    norm1 = math.sqrt(sum(value**2 for value in document1.values()))
    norm2 = math.sqrt(sum(value**2 for value in document2.values()))
    
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return dot_product / (norm1 * norm2)

#### Similarity matrix

Compute the similarity matrix between the documents of the corpus. While computing the similarities, you will record the two most similar documents that you will call `most_sim_doc1` and `most_sim_doc2`.

In [165]:
# Write your code here
docs = list(tfidf.keys())
n = len(docs)
matrix = [[0.0] * n for _ in range(n)]
max_score = -1
most_sim_docs1, most_sim_docs2 = None, None

for i, doc1 in enumerate(docs):
    for j, doc2 in enumerate(docs):
        score = cosine_similarity(tfidf[doc1], tfidf[doc2])
        matrix[i][j] = score
        if i < j and score > max_score:
            max_score = score
            most_sim_docs1, most_sim_docs2 = doc1, doc2

print(" " * 15, end="")
for doc in docs:
    print(f"{doc:15}", end="")
print()

# Print rows
for i, doc1 in enumerate(docs):
    print(f"{doc1:15}", end="")
    for j in range(n):
        print(f"{matrix[i][j]:15.3f}", end="")  # 3 decimals
    print()

               marbacka.txt   bannlyst.txt   jerusalem.txt  gosta.txt      nils.txt       herrgard.txt   osynliga.txt   troll.txt      kejsaren.txt   
marbacka.txt             1.000          0.037          0.005          0.080          0.085          0.004          0.093          0.147          0.071
bannlyst.txt             0.037          1.000          0.006          0.049          0.051          0.001          0.052          0.089          0.024
jerusalem.txt            0.005          0.006          1.000          0.004          0.005          0.371          0.028          0.007          0.002
gosta.txt                0.080          0.049          0.004          1.000          0.105          0.003          0.125          0.196          0.048
nils.txt                 0.085          0.051          0.005          0.105          1.000          0.005          0.111          0.188          0.050
herrgard.txt             0.004          0.001          0.371          0.003          0.005    

Give the name of the two novels that are the most similar.

In [167]:
print("Most similar:", most_sim_docs1, most_sim_docs2, "Similarity:", round(max_score, 3))

Most similar: jerusalem.txt herrgard.txt Similarity: 0.371


<h2>Turning in your assigment</h2>

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program, 
2. Read the text <i>Challenges in Building Large-Scale Information Retrieval Systems</i> about the history of <a href="https://research.google.com/people/jeff/WSDM09-keynote.pdf">Google indexing</a> by <a href="https://research.google.com/pubs/jeff.html">Jeff Dean</a> and write a short comment on it. See the details below.
3. Watch the speech delivered by Steve Jobs in 1985 (https://www.youtube.com/watch?v=NT6oUsn3ANA) and describe superficially in one paragraph how computers can amass knowledge and have dialogues with us.

You will submit your report as well as your notebook (for archiving purposes) to Canvas: <https://canvas.education.lu.se/>. You have two separate places: one of the report and the other for the program.

I suggest that you use this structure for your report:
1. Objectives and dataset
2. Method and program structure, where you should outline your program and possibly describe difficult parts. 
3. Results
4. Conclusion
5. Answer to possible questions

In your report of about two pages:
1. You will describe your indexer (Sect. Method) and comment your results (Sect. Result); in this description, you will write the regular expression you used for the tokenization (Sect. Method) and include the similarity matrix (Sect. Results);
2. In Jeff Dean's document, slide 45 describes an indexing technique. Tell how your index encoding is related to what Google did. (Sect. Other questions).
3. Referring to Jobs's speech, comment shortly how computers can amass knowledge and have dialogues with us.

To write your report, use Latex. This is mandatory as it will help you structure your text. You will then upload a PDF file in Canvas. You can try Overleaf (<a href="https://www.overleaf.com/">www.overleaf.com</a>) as it makes it easy to create Latex document.

The submission deadline is September 19, 2025. You will have only three submission attempts. The deadline for the second and third ones are one week after you are noticed of your result.